In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/NLP_and_CV_Projects/NLP/Hung')

Mounted at /content/drive


In [2]:
!nvidia-smi

Wed Jan 27 14:34:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **What are we going to do:**

- load the dataset from kaggle
- prepare the dataset and build a ``TextDataset``
- load the pre-trained GPT-2 model and tokenizer
- initialize ``Trainer`` with ``TrainingArguments``
- train and save the model
- test the model

In [3]:
!pip install transformers

     |████████████████████████████████| 1.8MB 8.2MB/s 
     |████████████████████████████████| 2.9MB 27.2MB/s 
     |████████████████████████████████| 890kB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e9551454f1cfd1788d2a20a9b55f4da23de7eed9c4d4c425bec2095de432c33f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
# !pip install wget

In [5]:
# import wget
# wget.download('https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/VNTQcorpus-big.txt')

After we uploaded the file with use `unzip` to extract the recipes.json. 

# Prepare the dataset and build a ``TextDataset``

The next step is to extract the instructions from all recipes and build a `TextDataset`. The `TextDataset` is a custom implementation of the [Pytroch `Dataset` class](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class) implemented by the transformers library. If you want to know more about Dataset in Pytroch you can check out this [youtube video](https://www.youtube.com/watch?v=PXOzkkB5eH0&ab_channel=PythonEngineer).

First, we are going to split the `recipes.json` into a `train` and `test` section and extract `Instructions` from the recipes and write them into a `train_dataset.txt` and `test_dataset.txt`

In [6]:
import torch
from transformers import AutoModel, AutoTokenizer

#phobert = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
custokenizer.add_tokens('\n')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


1

In [7]:
# def build_text_files(data, dest_path):
#     f = open(dest_path, 'w')
#     data_write = ''
#     for poem in data:
#         # summary = str(texts['Instructions']).strip()
#         # summary = re.sub(r"\s", " ", summary)
#         data_write += poem + "\n\n"
#     data_write = re.sub("\n\n+","\n\n", data_write)
#     f.write(data_write)

In [8]:
# import re
# import pandas as pd

# with open('all_data.txt') as f:
#     text = f.read()
# text = re.sub("\n3+","\n\n", text)
# text = re.sub("\n \n","\n\n", text)
# text = text.split("\n\n")


In [9]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(text,test_size=0.02) 

# build_text_files(train,'train_dataset.txt')
# build_text_files(test,'test_dataset.txt')

# print("Train dataset length: "+str(len(train)))
# print("Test dataset length: "+ str(len(test)))

In [10]:
# train_path = '/content/drive/My Drive/NLP_and_CV_Projects/NLP/Hung/68cleanregex.txt'
# test_path = '/content/drive/My Drive/NLP_and_CV_Projects/NLP/Hung/68cleanvl_test.txt'
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [11]:
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time

logger = logging.get_logger(__name__)
class CusTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=True,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    text = f.read()

                tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

                # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                i = 0
                while i < len(tokenized_text) - block_size + 1:
                    inds = tokenized_text[i : i + block_size]
                    for j in range(0, len(inds)):
                        if inds[j]==64001:
                            inds = inds[j+1:] #remove the first \n
                            break
                    for j in range(len(inds)-1, 0, -1):
                        if inds[j]==64001:
                            inds = inds[:j] #remove \n
                            break
                    i += len(inds)
                    self.examples.append(
                        tokenizer.build_inputs_with_special_tokens(inds)
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should loook for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [12]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = CusTextDataset(
          tokenizer=custokenizer,
          file_path=train_path,
          block_size=256)
     
    test_dataset = CusTextDataset(
          tokenizer=custokenizer,
          file_path=test_path,
          block_size=256)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=custokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,custokenizer)

In [13]:
len(test_dataset)

127

In [14]:
print(custokenizer.decode(train_dataset[1]))

<s> để cùng nhau múa giữa quê nghĩa tình 
 rom vong là điệu srók mình 
 là hình bóng sự yên bình quê ta 
 rom vong vui nhộn thiết tha 
 vọng từ trong srók vang ra mọi bề 
 rom vong điệu múa làng quê 
 khi xa thì nhớ khi về thì thương 
 rom vong là sợi tơ vương 
 buộc dân tộc với tình thương con người 
 rom vong tay múa môi cười 
 nhún vai chân bước theo người trước sau 
 rom vong đoàn kết đồng bào 
 sức khỏeđón năm mới vui tươi vàhạnh phúc 
 
 người đi giữ lấy khăn hồng 
 quê nhà chín nhớ mười mong em chờ 
 trở về bến cũ sông xưa 
 con đò khua nước ngẩn ngơ giữa dòng 
 sợ tan mất dải lụa hồng 
 mây chiều in bóng dòng sông lững lờ 
 nhớ câu chín đợi mười chờ 
 sóng trong lòng bỗng mấy bờ xôn xao 
 
 ăn quả nhớ người trồng cây 
 xướng vui hạnh phúc có ngày hôm nay 
 nhớ ơn công đức cao dày 
 của bao liệt sỹ tháng ngày trường sinh 
 có người tên tuổi quang vinh 
 nhiều người yên mộ một mình nơi nao 
 không khói hương không lời chào </s>


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [15]:
from transformers import Trainer, TrainingArguments, GPT2Config, GPT2LMHeadModel

#PRETRAINED GPT VNTQ

In [16]:
# vntq = GPT2LMHeadModel.from_pretrained('./VNTQcorpus/checkpoint-72000')

In [17]:
# vntq

In [18]:
# vntq.lm_head.weight

Re-create weight FC layer to fine-tuning

In [19]:
# rand_weight = torch.rand(vntq.lm_head.weight.shape)

In [20]:
# vntq.lm_head.weight = torch.nn.parameter.Parameter(rand_weight)

In [21]:
# vntq.lm_head.weight

In [22]:
# vntq.config

# NEW MODEL

In [23]:
# task = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size

In [24]:
# configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
#                            task_specific_params=task,
#                            eos_token_id=custokenizer.eos_token_id,
#                            bos_token_id=custokenizer.bos_token_id, 
#                            pad_token_id=custokenizer.pad_token_id,
#                            sep_token_id=custokenizer.sep_token_id
#                            )
# poem = GPT2LMHeadModel(configuration)
# poem

In [25]:
# poem.config

Copying weight from VNQT trained model to the new one

In [26]:
# poem.load_state_dict(vntq.state_dict())

In [27]:
poem = GPT2LMHeadModel.from_pretrained('./poems/checkpoint-70000')

In [28]:
from transformers.trainer_callback import TrainerCallback
from transformers import pipeline
class PrinterCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if int(state.epoch)%10==0:
            pipe = pipeline('text-generation', model=poem, tokenizer=custokenizer, device=0)
            with open("/content/drive/MyDrive/NLP_and_CV_Projects/NLP/Hung/poems/sample.txt", "a") as f:
                f.write(pipe('<s> tìm về một thuở hạ xưa')[0]['generated_text'])
                f.write("\n===========================================\n")
                f.close()

In [29]:
training_args = TrainingArguments(
    output_dir="./poems", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    save_steps=10000, # after # steps model is saved 
    save_total_limit = 2, # delete other checkpoints
    warmup_steps=500,    # number of warmup steps for learning rate scheduler
    logging_dir='./poems/logs', # directory for storing logs
    logging_steps=1000,
    prediction_loss_only=True,
    )

In [30]:
trainer = Trainer(
    model=poem,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks = [PrinterCallback],
)

# Train and save the model

To train the model we can simply run `Trainer.train()`.

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [31]:
trainer.train(model_path = './poems/checkpoint-70000')
# trainer.train()
trainer.save_model()

Step,Training Loss
71000,0.003300
72000,0.223200
73000,0.217700
74000,0.211400
75000,0.206500
76000,0.201200
77000,0.196500
78000,0.192600
79000,0.188300
80000,0.184200


# Test the model

To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [32]:
from transformers import pipeline

poem = pipeline('text-generation', model="./poems", tokenizer=custokenizer, device=0)

In [33]:
# f.close()

In [34]:
# i = 0
# with open("./poems/fake.txt", "a") as f:
#   while i < len(text):
#     tho = poem('<s> ')[0]['generated_text']
#     tho = re.sub("<s> ","", tho)
#     tho = re.sub(" </s>","\n\n", tho)
#     f.write(tho)
#   i += 1

In [35]:
a = poem('<s> mùa xuân')
print(a[0]['generated_text'])

<s> mùa xuân cắp rổ ra đồng vớt hoa 
 
 tác giả lê danh tuệ đt 
 học viện chính trị bộ qp ngô quyền 
 
 hai mươi năm đã đi qua 
 mà lòng con vẫn nhớ cha không rời 
 người đi từ biệt cõi đời 
 rưng rưng ngấn lệ nghẹn lời tiễn đưa 
 một đời đèn sách nắng mưa 
 nhặt gom con chữ kế thừa cha ông 
 giữ gìn tâm đức sáng trong 
 gia phong nề nếp nối dòng tổ tiên 
 bà con xóm dưới làng trên 
 giúp nhau tắt lửa tối đèn hôm mai 
 trải qua tháng rộng năm dài 
 dạy con dạy cháu vui chơi học hành 
 vợi vời một khoảng trời xanh 
 đức nhân như của để dành cháu con 
 ơn cha dưỡng dục vuông tròn 
 tình cha để lại mãi còn ghi sâu 
 
 thương sau mấy ngọn lục bình 
 nước trôi xuôi ngược bập bềnh nơi đâu 
 hoàng hôn đã phủ giang đầu 
 tiếng chim lẻ bạn gọi nhau não lòng 
 xứ người lưu lạc bao năm 
 thương câu lục bát muôn ngàn dặm xa 
 thầm thì mấy khúc dân ca 
 mây trôi bèo dạt phận hoa bến đời 
 lục bình xuôi ngược mênh mang 
 đời người mấy chuyến đò ngang sẽ dừng 
 
 cho gươm mời đến thúc lang


In [36]:
b = poem('<s> anh')
print(b[0]['generated_text'])

<s> anh về quê mẹ em không 
 tình quê nặng nghĩa tấm lòng mẹ cha 
 
 bây giờ xã đã thành phường 
 xóm thôn đánh số tên đường làm sang 
 bây giờ phố đã thay làng 
 người ta bán đất vội vàng ăn chơi 
 những cô thôn nữ phố tôi 
 ai phôn tạo dáng hay cười làm duyên 
 phấn son nhan sắc được quyền 
 dự thi hoa hậu khắp miền mà vui 
 tây tàu du lịch khắp nơi 
 những cô thôn nữ phố tôi cũng tài 
 
 mè đen trắng cũng là vừng 
 họ rằng sai vẫn lừng khừng nói im 
 vấp mơ hồ rớt niềm tin 
 vàng thau lẫn lộn biết tìm được không 
 nghe thật thà rõ sâu nông 
 thước nào đo thấu đáy lòng dối gian 
 người khôn chạy trước giành ngoan 
 mình khờ vác dại tụt đoàn đua sau 
 dại khôn nào tránh buồn đau 
 kiếm hoài chẳng được mình cầu tỉnh say 
 
 sáu năm vắng mẹ mẹ ơi 
 mẹ đi về chốn phương trời xa xôi 
 con đà tủi phận mồ côi 
 mất cha từ lúc chào đời xót xa 
 mẹ vừa là mẹ là cha 
 nuôi con khôn lớn thật là thương thay 
 biết bao vất vả đắng cay 
 thức khuya dậy sớm đó đây tảo tần 
 lo cho con có cơm ăn
